In [7]:
from scipy.stats import shapiro, anderson, kstest, ttest_ind, mannwhitneyu, ks_2samp, rankdata
import math
from statistics import variance, mean

In [50]:
# read results from our final test runs

hc, metropolis, sa = [], [], []
log_file = open("final_contest.tsv")

for line in log_file:

    temp = line.split("\t")
    if temp[0] == "hc_maxIterations":
        hc.append(float(temp[3]))
    elif temp[0] == "metropolis_maxIterations":
        metropolis.append(float(temp[3]))
    elif temp[0] == "simulated_annealing_maxIterations":
        sa.append(float(temp[3]))

log_file.close()


hc.sort()
metropolis.sort()
sa.sort()

In [52]:
# print basic descriptive statistics

def describe(seq:list, name:str):
    print(f"Stats for {name}:")
    print(f"Average Costs: {mean(seq)}\nLowest Cost Solution: {min(seq)}\nStd.Deviation: {math.sqrt(variance(seq))}\n\n")

describe(hc, "Hill Climber")
describe(sa, "Simulated Annealing")
describe(metropolis, "Metropolis")

Stats for Hill Climber:
Average Costs: 1.5799999999999998
Lowest Cost Solution: 1.2
Std.Deviation: 0.22010098692292232


Stats for Simulated Annealing:
Average Costs: 1.3399999999999999
Lowest Cost Solution: 1.2
Std.Deviation: 0.16465452046971288


Stats for Metropolis:
Average Costs: 1.22
Lowest Cost Solution: 1.2
Std.Deviation: 0.06324555320336757




In [11]:
# test for normal distribution

def test_normal(seq):

        a_shapiro = shapiro(seq)
        print("{:30s} {:30f}".format("Shapiro-Wilk: p-value", a_shapiro[1]))

        # Kolmogorov-Smirnov
        a_kolmogorov = kstest(seq, 'norm')
        print("{:30s} {:30f}".format("Kolmogorov-Smirnov: p-value", a_kolmogorov[1]))

        # Anderson (assumes alpha = 0.05)
        a_anderson = anderson(seq, dist='norm')
        print("{:30s} {:30f}".format("Anderson: test statistic", a_anderson[0]))
        print("{:30s} {:30f}".format("Anderson: critical value", a_anderson[1][2]),"\n")

test_normal(hc)
test_normal(sa)
test_normal(metropolis)

# >> data is not normal distributed !


Shapiro-Wilk: p-value                                0.009139
Kolmogorov-Smirnov: p-value                          0.000000
Anderson: test statistic                             0.990038
Anderson: critical value                             0.684000 

Shapiro-Wilk: p-value                                0.008500
Kolmogorov-Smirnov: p-value                          0.000000
Anderson: test statistic                             0.947045
Anderson: critical value                             0.684000 

Shapiro-Wilk: p-value                                0.000000
Kolmogorov-Smirnov: p-value                          0.000000
Anderson: test statistic                             3.208057
Anderson: critical value                             0.684000 



In [54]:
# mann whitney u test for unpaired and not normal distributed data 

def mwu(seq1, seq2, alpha):

    mannwhitney = mannwhitneyu(seq1, seq2, alternative="two-sided")
    print("{:30s} {:30f}".format("p-value", mannwhitney[1]))

    if mannwhitney[1] < alpha:
        print(f"seq1 is not significantly different to seq2 for alpha={alpha}")
    else:
        print(f"seq1 is significantly different to seq2 for alpha={alpha}2")

print("\nMann - Whitney U test/ Wilcoxon rank-sum test\n")

print("Hill Cimber - Metropolis")
mwu(hc, metropolis, 0.05)

print("\nHill Climber - Simulated Annealing")
mwu(hc, sa, 0.05)

print("\nSimulated Annealing - Metropolis")
mwu(sa, metropolis, 0.05)


Mann - Whitney U test/ Wilcoxon rank-sum test

Hill Cimber - Metropolis
p-value                                              0.001279
seq1 is not significantly different to seq2 for alpha=0.05

Hill Climber - Simulated Annealing
p-value                                              0.019390
seq1 is not significantly different to seq2 for alpha=0.05

Simulated Annealing - Metropolis
p-value                                              0.054522
seq1 is significantly different to seq2 for alpha=0.052


In [55]:
# vargha & Delaney Effect Measure

def vd_measure(seq1, seq2):
    c = list(seq1) + list(seq2)
    c_ranked = rankdata(c, method='average')
    a_ranked = list(c_ranked[:len(seq1)])
    b_ranked = list(c_ranked[len(seq1):len(seq1) + len(seq2)])
    ranksum_a = sum(a_ranked)
    ranksum_b = sum(b_ranked)

    print("\nVargha’s and Delaney’s A Measure")
    print("0.5=no, 0.56=small, 0.64=medium, 0.71=big effect")
    A = 1 / len(seq2) * (ranksum_a / len(seq1) - (len(seq1) + 1) / 2)
    if A < 0.5:
        A = 1 - A
    print("{:30s} {:30f}".format("A measure", A))


print("Hill Cimber - Metropolis")
vd_measure(hc, metropolis)

print("\nHill Climber - Simulated Annealing")
vd_measure(hc, sa)

print("\nSimulated Annealing - Metropolis")
vd_measure(sa, metropolis)


Hill Cimber - Metropolis

Vargha’s and Delaney’s A Measure
0.5=no, 0.56=small, 0.64=medium, 0.71=big effect
A measure                                            0.890000

Hill Climber - Simulated Annealing

Vargha’s and Delaney’s A Measure
0.5=no, 0.56=small, 0.64=medium, 0.71=big effect
A measure                                            0.800000

Simulated Annealing - Metropolis

Vargha’s and Delaney’s A Measure
0.5=no, 0.56=small, 0.64=medium, 0.71=big effect
A measure                                            0.710000
